In [1]:
# import modules
import os,binascii
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from aesLongKeyGen24 import *

In [2]:
# brute force attack to find 20 bit key
def bruteForceAttack(plainText, cipherText):
    iv=b'\0\0\0\0\0\0\0\0\0\0\0\0\0\0\0\0'
    plainText = plainText.encode('utf-8')
    for shortKey in range(0, 2**20):
        shortKey = shortKey << 4
        shortKey = bytearray(shortKey.to_bytes(3, byteorder='big'))
        longKey = expandKey(shortKey)
        cipher = Cipher(algorithms.AES(longKey), modes.CBC(iv))
        encryptor = cipher.encryptor()
        encryptedText = encryptor.update(plainText) + encryptor.finalize()
        if encryptedText.hex() == cipherText:
            return shortKey
    return -1

In [3]:
# read aesPlaintexts.txt and aesCiphertexts.txt
with open('aesPlaintexts.txt', 'r') as f:
    plainTexts = f.read().splitlines()
with open('aesCiphertexts.txt', 'r') as f:
    cipherTexts = f.read().splitlines()
print(f'Plaintexts: {plainTexts}')
print(f'Ciphertexts: {cipherTexts}')

Plaintexts: ['Counterclockwise', 'sonicthehedgehog', 'TheDeterminantor', 'FeedbackRegister']
Ciphertexts: ['fe0f42ae809fe1e2ff5b59725ef52048', 'ab4e40c6bf551a4d0794c0fd65074003', 'ffc7bb77f95466128a61f4ad9916b8ca', 'd9ff735a89509dc5c23d2eb27cf00904', 'ca6889853e3ddfaf621b87ee4966e274']


In [4]:
# Tried to find key from 1st plaintext and ciphertext
key = bruteForceAttack(plainTexts[0], cipherTexts[0])
if key == -1:
    print("Key not found")
else:
    print("Key found: " + str(key.hex()))

Key found: 8e6330


In [5]:
# find secret plaintext by decrypting corresponding ciphertext with key
# do the following if key is found
sCt = cipherTexts[-1]
sKey = bytearray.fromhex(key.hex())
lKey = expandKey(sKey)
iv=b'\0\0\0\0\0\0\0\0\0\0\0\0\0\0\0\0'
cipher = Cipher(algorithms.AES(lKey), modes.CBC(iv))
decryptor = cipher.decryptor()
sPt = decryptor.update(bytearray.fromhex(sCt)) + decryptor.finalize()
print(f'Secret plaintext: {sPt.decode("utf-8")}')

# write key, expandedKey and secret plaintext to secretInfo.txt
with open('secretInfo.txt', 'w') as f:
    f.write(f'Key: {key.hex()}\n')
    f.write(f'Expanded key: {lKey.hex()}\n')
    f.write(f'Secret plaintext: {sPt.decode("utf-8")}\n')

Secret plaintext: mediumaquamarine


In [6]:
# find keys for all plaintexts and ciphertexts
keys = []
for i in range(len(plainTexts)):
    shortKey = bruteForceAttack(plainTexts[i], cipherTexts[i])
    keys.append(shortKey.hex())
    if (shortKey == -1):
        print("short key not found")
    else:
        print(f'for plaintext:{plainTexts[i]} and ciphertext:{cipherTexts[i]}')
        print("short key found: ", shortKey.hex())

print(f'Keys: {keys}')

for plaintext:Counterclockwise and ciphertext:fe0f42ae809fe1e2ff5b59725ef52048
short key found:  8e6330
for plaintext:sonicthehedgehog and ciphertext:ab4e40c6bf551a4d0794c0fd65074003
short key found:  8e6330
for plaintext:TheDeterminantor and ciphertext:ffc7bb77f95466128a61f4ad9916b8ca
short key found:  8e6330
for plaintext:FeedbackRegister and ciphertext:d9ff735a89509dc5c23d2eb27cf00904
short key found:  8e6330
Keys: ['8e6330', '8e6330', '8e6330', '8e6330']
